In [2]:
"""Main Travel Assistant application"""
import tkinter as tk
from storage import *
from pages import *
from packing_GUI import PackingListFrame
from file_handler import *
from packing_func import get_items
from emergency_contact_app import EmergencyContactModule

class TravelAssistantApp:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("Travel Assistant - Trip Planner")
        self.root.geometry("1200x750")
        
        self.bg_color = "#f0f0f0"
        self.frame_bg = "#ffffff"
        self.button_bg = "#4a6fa5"
        self.button_fg = "white"
        self.title_bg = "#2c3e50"
        
        # StorageManager auto-loads on creation
        self.storage = StorageManager()
        
        # Main frames
        self.main_frame = tk.Frame(self.root, bg=self.bg_color)
        self.itinerary_frame = tk.Frame(self.root, bg=self.bg_color)
        self.emergency_container = None
        self.emergency_module = None
        
        # Initialize pages
        self.add_trip_page = AddTripPage(self.root, self.storage, self.bg_color,
                                        self.frame_bg, self.button_bg, self.button_fg,
                                        self.title_bg, self.back_to_itinerary)
        
        self.add_accommodation_page = AddAccommodationPage(self.root, self.storage, self.bg_color,
                                                          self.frame_bg, self.button_bg, self.button_fg,
                                                          self.title_bg, self.back_to_itinerary)
        
        self.add_activities_page = AddActivitiesPage(self.root, self.storage, self.bg_color,
                                                    self.frame_bg, self.button_bg, self.button_fg,
                                                    self.title_bg, self.back_to_itinerary)
        
        self.summary_page = SummaryPage(self.root, self.storage, self.bg_color,
                                       self.frame_bg, self.button_bg, self.button_fg,
                                       self.title_bg, self.back_to_itinerary)

        #-------- Initialize Packing List Page ------
        # Create an instance of the PackingListFrame class and assign it to self.packing_page.
        # This sets up the packing list GUI with all its elements (inputs, buttons, listbox, progress bar)
        # and passes in the necessary configuration colors and the callback function to return to the main menu.
        self.packing_page = PackingListFrame(
            self.root, # parent window (main Tk root)
            self.bg_color, # main background color
            self.frame_bg, # left/right panel background color
            self.button_bg, # button background color
            self.button_fg, # button text color
            self.title_bg, # title bar background color
            self.back_to_main # callback function to return to main menu
        )
        
        self._setup_main_menu()
        self._setup_itinerary_menu()
        
        self.main_frame.pack(fill="both", expand=True)
        
        # Auto-save on window close
        self.root.protocol("WM_DELETE_WINDOW", self.on_closing)
    
    def on_closing(self):
        """Auto-save when closing the app"""
        self.storage.auto_save()
        # Save emergency contacts if module was initialized
        if self.emergency_module:
            self.emergency_module.save_data()
        self.root.destroy()
    
    def _setup_main_menu(self):
        tk.Label(self.main_frame, text="✈️ Travel Assistant", font=("Arial", 24, "bold"), 
                bg=self.title_bg, fg="white", padx=20, pady=10).pack(fill="x", pady=(0, 20))

        button_frame = tk.Frame(self.main_frame, bg=self.bg_color)
        button_frame.pack(pady=20)

        buttons = [
            ("📅 Itinerary Builder", self.show_itinerary),
            ("🎒 Packing List Generator", self.show_packing_list),
            ("📞 Emergency Contact", self.show_emergency_contact),
            ("❌ Exit", self.on_closing)
        ]

        for text, command in buttons:
            btn = tk.Button(button_frame, text=text, width=25, height=2,
                          font=("Arial", 11), bg=self.button_bg, fg=self.button_fg,
                          command=command, cursor="hand2")
            btn.pack(pady=8)
    
    def _setup_itinerary_menu(self):
        tk.Label(self.itinerary_frame, text="📅 Itinerary Builder", font=("Arial", 20, "bold"), 
                bg=self.title_bg, fg="white", padx=20, pady=10).pack(fill="x", pady=(0, 20))

        itinerary_buttons = tk.Frame(self.itinerary_frame, bg=self.bg_color)
        itinerary_buttons.pack(pady=20)

        itinerary_btn_configs = [
            ("➕ Add New Trip", self.show_add_trip),
            ("🏨 Add Accommodation", self.show_add_accommodation),
            ("🎭 Add Activities", self.show_add_activities),
            ("📋 View Itinerary Summary", self.show_summary),
            ("🔙 Back to Main Menu", self.back_to_main)
        ]

        for text, command in itinerary_btn_configs:
            btn = tk.Button(itinerary_buttons, text=text, width=30, height=2,
                          font=("Arial", 10), bg=self.button_bg, fg=self.button_fg,
                          command=command, cursor="hand2")
            btn.pack(pady=6)
    
    def show_packing_list(self):
        """
        Show the Packing List page.
    
        1. Call _hide_all_pages() to hide any other open pages in the GUI.
        2. Call the show() method of the PackingListFrame instance (self.packing_page)
           to display the packing list interface to the user.
        """
        self._hide_all_pages() 
        self.packing_page.show()

    def show_emergency_contact(self):
        """Show emergency contact manager"""
        self._hide_all_pages()
        
        # Create container frame if not exists
        if not self.emergency_container:
            self.emergency_container = tk.Frame(self.root, bg=self.bg_color)
        
        # Initialize or refresh the emergency contact module
        self.emergency_module = EmergencyContactModule(
            self.emergency_container,
            self.bg_color,
            self.frame_bg,
            self.button_bg,
            self.button_fg,
            self.title_bg,
            self.back_to_main
        )
        
        # Show the container
        self.emergency_container.pack(fill="both", expand=True)
    
    # Navigation methods
    def show_itinerary(self):
        self._hide_all_pages()
        self.itinerary_frame.pack(fill="both", expand=True)
    
    def show_add_trip(self):
        self._hide_all_pages()
        self.add_trip_page.refresh()
        self.add_trip_page.show()
    
    def show_add_accommodation(self):
        self._hide_all_pages()
        self.add_accommodation_page.refresh()
        self.add_accommodation_page.show()
    
    def show_add_activities(self):
        self._hide_all_pages()
        self.add_activities_page.refresh()
        self.add_activities_page.show()
    
    def show_summary(self):
        self._hide_all_pages()
        self.summary_page.refresh()
        self.summary_page.show()
    
    def back_to_main(self):
        self._hide_all_pages()
        self.main_frame.pack(fill="both", expand=True)
    
    def back_to_itinerary(self):
        self._hide_all_pages()
        self.itinerary_frame.pack(fill="both", expand=True)
    
    def _hide_all_pages(self):
        """Hide all frames/pages"""
        self.main_frame.pack_forget()
        self.itinerary_frame.pack_forget()
        
        if self.emergency_container:
            self.emergency_container.pack_forget()
        
        # Hide other pages if they have hide method
        for page in [self.add_trip_page, self.add_accommodation_page, 
                    self.add_activities_page, self.summary_page]:
            if hasattr(page, 'hide'):
                page.hide()

        #Hide Packing list page
        # Check if the attribute 'packing_page' exists for this object. if hasattr........
        # If it does, call its hide() method to remove the packing list GUI from view. self.packing.....
        # Without hasattr, the program could crash if packing_page hasn’t been initialized yet.
        if hasattr(self, "packing_page"):
                self.packing_page.hide()
    
    def run(self):
        self.root.mainloop()


# Main execution
if __name__ == "__main__":
    app = TravelAssistantApp()
    app.run()

Auto-loaded 2 trip(s) from travel_data.txt
Auto-loaded 2 trip(s) from travel_data.txt
Auto-saved 1 trip(s) to travel_data.txt
